<br>
<b> Importing all of the needed packages here. <br>
<br>

In [1]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
import urllib
from io import BytesIO
from io import StringIO
import json

C:\Users\brian\Anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


<br>
<b> Let's get started! </b>
<br></br>

We will first define functions to get the data and put it into dataframe

In [2]:
# The dataset link
# This is a zip file.

link = r'https://storage.googleapis.com/ds--tasks-datasets/renewable-energy.zip'

In [3]:
# a function to extract zipped files with text files
def get_zip_files(link):
    # get the zip file
    url = urllib.request.urlopen(link).read()
    files = ZipFile(BytesIO(url))
    
    # get the file names into a list; you could use this list for different things 
    # and to get certain files etc.
    listOfFilesNames = files.namelist()
    
    # open a file from the list; in this case position 23.  This part could be a loop etc. 
    # and if 'specificFileName' do something different etc.
    articles = {}
    for i in listOfFilesNames:
        if i.endswith('.txt') and i.startswith('renewable'):
            names = 'articleID' + i.replace('renewable-energy/','').replace('.txt','')
            txt = str(files.open(i).read().decode("utf-8"))
            articles[names] = txt

    # return the article as a dictionary to identify the article by the id.
    # The key format is 'articleID#'  --->> # represents integer numbers
    return articles

In [4]:
articles = get_zip_files(link)

<br></br>
Let us look at the keys.
<br></br>

In [5]:
articles.keys()

dict_keys(['articleID29', 'articleID15', 'articleID14', 'articleID28', 'articleID16', 'articleID17', 'articleID13', 'articleID12', 'articleID10', 'articleID11', 'articleID9', 'articleID8', 'articleID5', 'articleID4', 'articleID6', 'articleID7', 'articleID3', 'articleID2', 'articleID1', 'articleID20', 'articleID21', 'articleID23', 'articleID22', 'articleID26', 'articleID27', 'articleID25', 'articleID19', 'articleID18', 'articleID24', 'articleID30'])

<br></br>
Here, the articles have the 'articleID#' as a key with paragraphs with 'n'.
By putting the keys into the df, we can read the articles.
Let's look at the article #1.
<br></br>

In [6]:
articles['articleID1']

'## Solar Power Gets a Boost: Efficiency and Material Advancements\n\nSolar panels are becoming increasingly powerful and accessible thanks to exciting advancements in two key areas: efficiency and materials. \n\n**Efficiency improvements** are pushing the boundaries of how much sunlight can be converted into electricity. Researchers are constantly refining existing technologies and exploring new designs. One example is the development of **bifacial solar panels**, which capture sunlight reflected off the ground from both sides, boosting energy production by up to 20%. Additionally, **improved cell design and manufacturing processes** are leading to higher quality and more efficient solar cells.\n\nThe future of solar also lies in **new materials**. **Perovskites** are a promising alternative to traditional silicon, offering the potential for even higher efficiency and lower manufacturing costs. Additionally, researchers are exploring the use of **organic semiconductors**, which can be

<br></br>
As you can see, we can tidy up the texts. We will write a function to extract the title as a label, which is the first element of the list and the turn paragraphs into sentences without the '\n' and 'b' in front of the sentences. Finally, we will put it into the pd.DataFrame.
<br></br>

In [7]:
# transferring dictionary to DataFrame. 
def Dict2df(article_dict):
    labels = {}
    values_fixed = {}
    for keys,values in article_dict.items():
        sentences = values.split('\n')
        labels[keys] = sentences[0].replace('## ','')
        #values_fixed[keys] = [i.replace('*','').replace('**','').replace('* **', '') for i in sentences[1:] if i != '']
        values_fixed[keys] = ' '.join([i.replace('*','').replace('**','').replace('* **', '') for i in sentences[1:] if i != ''])
    
    df_x = pd.DataFrame(values_fixed.items(), columns=['ID', 'Paragraph'])
    df_y = pd.DataFrame(labels.items(), columns=['ID', 'Title'])
    df = pd.concat([df_x,df_y['Title']],axis=1,join='inner').set_index('ID')
    return df

In [8]:
df = Dict2df(articles)

In [9]:
df.head()

,Paragraph,Title
ID,,
articleID29,The world is witnessing a surge in renewable e...,"Global Renewable Energy Adoption: Trends, Chal..."
articleID15,"Bioenergy, harnessing the power of organic mat...",Bioenergy Blossoms: Advancing Biofuels and Bio...
articleID14,"Bioenergy, the fuel derived from organic matte...",Bioenergy Blossoms: Advancements in Biofuels a...
articleID28,The global energy landscape is undergoing a si...,Global Trends in Renewable Energy Adoption: A ...
articleID16,The quest for efficient and reliable energy st...,Powering the Future: Innovation in Battery Tec...


<br></br>
The dataframe has article id as the index and two columns for paragraphs and titles.  
Now, we do not have to fine-tune the model as specified by the challenge objectives. We can go ahead with the pipeline using pre-trained models for extracting the key information from the article. The model we will use is the distilbart-cnn-12-6 by sshleifer.
<br></br>

In [10]:
from transformers import pipeline
summarizer = pipeline("summarization",model='sshleifer/distilbart-cnn-12-6')

<br></br>
We can try articleID20 for an example.
<br></br>

<b> Here is the article: </b>

In [11]:
'summarize: ' + df.loc['articleID20']['Paragraph']

"summarize: The transition to renewable energy requires a smarter grid, and innovative technologies are weaving renewables seamlessly into the energy fabric. Here's how smart meters and energy management, coupled with advancements in grid stability, are paving the way. Smart meters leading the charge:  These intelligent devices go beyond simply measuring electricity consumption. They:  Provide real-time data: Empowering consumers to monitor their energy usage and adjust consumption patterns based on grid needs and dynamic pricing.  Enable two-way communication: Facilitating real-time communication between consumers and utilities, allowing for demand response programs and grid optimization. Energy management gets smarter:  Smart grids integrate with home energy management systems, allowing for:  Automated load control: Appliances and devices can be automatically shifted to off-peak hours, reducing peak demand and grid stress.  Integration of distributed generation: Homes with rooftop so

<b> Here is the key sentence: </b>

In [12]:
summarizer(df.loc['articleID20']['Paragraph'],min_length=5,max_length=18)

[{'summary_text': ' The transition to renewable energy requires a smarter grid . Smart meters and energy management'}]

<br>
<br>
<i> *** note for myself for the future: I should have added 'summarize: ' in front of the paragraph to specifically tell the model that I am summarizing </i>
<br>
<br>

<br></br>
Now, we can write a function to extract a key sentence by inputting the article id. 

In [13]:
def extractKeySentences(articleId):
    summary = summarizer(df.loc[articleId]['Paragraph'],min_length=5,max_length=18)
    print(summary)
    summary_json = json.dumps(summary,indent = 4)
    return summary_json

In [14]:
summary = extractKeySentences('articleID20')

[{'summary_text': ' The transition to renewable energy requires a smarter grid . Smart meters and energy management'}]


<br></br>
As you can see, the summary text will return in JSON format. In the future, we can use different models or fine-tune the pre-trained model as well as experiment with lengths to get a better key sentence. 
<br></br>

<br></br>
<b> We can go ahead with the second objective: the comparison of the two articles. </b><br>
Here, we can go two ways. <br>
1. We can extract the key sentences and compare those two.
2. We can extract each sentence of document 1 and compare to the document 2.
    
The objective of the challenge states that we are comparing two articles, which means we will go ahead and choose the second option.


We will load the pre-trained model (sentence-transformer/msmarco-distilbert-cos-v5) and write functions for encoding, mean pooling, and the compareArticle to look at similarities and dissimilarities. In the end, the returned format will be in JSON format as well as specified in the objectives. 

In [15]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/msmarco-distilbert-cos-v5")
model = AutoModel.from_pretrained("sentence-transformers/msmarco-distilbert-cos-v5")
    

#Mean Pooling - Take average of all tokens
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


#Encode text
def encode(texts):
    # Tokenize sentences
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input, return_dict=True)

    # Perform pooling
    embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    embeddings = F.normalize(embeddings, p=2, dim=1)
    
    return embeddings

# Here, we will compare doc1 to doc2 by taking one sentence of doc1 and looking at similarity in doc2 sentences.
# Then, we will pick one most similar and dissimilar to the sentence.
# We will put those in the JSON format as well. 
def compareArticles(articleId1, articleId2):
    Doc1 = df.loc[articleId1]['Paragraph'].split('.')[:-1]
    Doc2 = df.loc[articleId2]['Paragraph'].split('.')[:-1]
    len1 = len(Doc1)
    comparisonSummary = {}
    for i in range(len1):
        # Sentences we want sentence embeddings for
        sentence1 = Doc1[i]
        
        #Encode query and docs
        sent_emb = encode(sentence1)
        doc_emb = encode(Doc2)

        #Compute dot score between query and all document embeddings
        scores = torch.mm(sent_emb, doc_emb.transpose(0, 1))[0].cpu().tolist()
        #Combine docs & scores
        doc_score_pairs = list(zip(scores, Doc2))
        
        #Sort by decreasing score
        doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[0], reverse=True)

        #Put it into the dictionary
        comparisonSummary[i] = (sentence1,doc_score_pairs[0],doc_score_pairs[-1])

    #Turn the dictionary into the JSON format
    comparisonSummary_json = json.dumps(comparisonSummary,indent = 4)
    return comparisonSummary_json

In [16]:
summary = compareArticles('articleID1','articleID22')

<br></br>
Now, we can see the summary by loading the summary with json.load and see what is similar and dissimilar sentences in doc2 compared to doc1. 
<br></br>

In [17]:
summary = json.loads(summary)

In [18]:
for i in range(len(summary)):
    i = str(i)
    print('sentence of doc 1: ', summary[i][0])
    print('most similar sentence in doc2 [score, sentence]: ', summary[i][1])
    print('most dissimilar sentence in doc2 [score, sentence]: ', summary[i][2])
    print('\n')

sentence of doc 1:  Solar panels are becoming increasingly powerful and accessible thanks to exciting advancements in two key areas: efficiency and materials
most similar sentence in doc2 [score, sentence]:  [0.4124220609664917, 'The transition to renewable energy demands robust policies']
most dissimilar sentence in doc2 [score, sentence]:  [0.16970928013324738, '  International Renewable Energy Agency (IRENA): This intergovernmental organization works towards the widespread adoption of renewable energy by providing assistance and facilitating collaboration between countries']


sentence of doc 1:    Efficiency improvements are pushing the boundaries of how much sunlight can be converted into electricity
most similar sentence in doc2 [score, sentence]:  [0.4276307225227356, 'The transition to renewable energy demands robust policies']
most dissimilar sentence in doc2 [score, sentence]:  [0.1575373113155365, '  Renewable energy certificates (RECs): These tradable instruments allow enti

<br></br>
Great! These implementation displays the key sentences as well as comparison of two articles. It is very flexiable to change and add more functionalities upon this. 

<br>
<br>
We can try to fine-tune the text.
<br>
<br>
We have to get x and y. 
<br>
<br>

In [19]:
df.loc['articleID1']['Title']

'Solar Power Gets a Boost: Efficiency and Material Advancements'

In [20]:
df.loc['articleID1']['Paragraph']

'Solar panels are becoming increasingly powerful and accessible thanks to exciting advancements in two key areas: efficiency and materials.  Efficiency improvements are pushing the boundaries of how much sunlight can be converted into electricity. Researchers are constantly refining existing technologies and exploring new designs. One example is the development of bifacial solar panels, which capture sunlight reflected off the ground from both sides, boosting energy production by up to 20%. Additionally, improved cell design and manufacturing processes are leading to higher quality and more efficient solar cells. The future of solar also lies in new materials. Perovskites are a promising alternative to traditional silicon, offering the potential for even higher efficiency and lower manufacturing costs. Additionally, researchers are exploring the use of organic semiconductors, which can be lightweight, flexible, and even printed onto various surfaces. These advancements could pave the w